# Silver Layer Transformation - Circuit Info

## Objective
Transform the `bronze.circuit_info` table into a clean and standardized Silver table.

## Steps
1. **Read** data from the Bronze table `bronze.circuit_info`.
2. **Select & Rename** relevant columns:
   - Keep: `circuit_name`, `circuit_location`, `circuit_country`, `distance`, `angle`
   - Standardize: `year`, `round`, `session_key`
   - Drop: `raw`, `idx`, `record_type`, `number`, `name` (redundant/unnecessary)
3. **Data Cleaning**
   - Cast numeric fields (`year`, `round`, `distance`, `angle`) to correct types.
   - Trim strings to remove extra spaces.
4. **Write** the cleaned data into the Silver schema as `silver.circuit_info`.

## Output
- A clean, deduplicated, and standardized Silver table:  
  `silver.circuit_info`


In [0]:
from pyspark.sql import functions as F

# Read from Bronze
circuit_info_bronze = spark.table("bronze.circuit_info")

# Transformations
circuit_info_silver = (
    circuit_info_bronze
    .select(
        F.col("circuit_name").alias("circuit_name"),
        F.col("circuit_location").alias("circuit_location"),
        F.col("circuit_country").alias("circuit_country"),
        F.col("distance").cast("double").alias("distance"),
        F.col("angle").cast("double").alias("angle"),
        F.col("year").cast("int").alias("year"),
        F.col("round").cast("int").alias("round"),
        F.col("session_key").cast("string").alias("session_key")
    )
    .withColumn("circuit_name", F.trim(F.col("circuit_name")))
    .withColumn("circuit_location", F.trim(F.col("circuit_location")))
    .withColumn("circuit_country", F.trim(F.col("circuit_country")))
    .dropDuplicates()
)

# Write to Silver
(
    circuit_info_silver
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable("silver.circuit_info")
)